# 1. Import and Install Dependencies

In [2]:
# !pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [ ]:
#test

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [4]:
from config import no_sequences, sequence_length, actions, DATA_PATH

# 2. Keypoints using MP Holistic

In [5]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [6]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [7]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

# 3. Extract Keypoint Values

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh, face])

# 4. Setup Folders for Collection

In [9]:
# create folders

for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [16]:
# create folders
VIDEO_PATH = os.path.join('videos')

for action in actions:

    try:
        os.makedirs(os.path.join(VIDEO_PATH, action))
    except:
        pass

In [11]:
VIDEO_PATH

'videos'

# 5. Collect Keypoint Values for Training and Testing

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [17]:
cap = cv2.VideoCapture(1)
# Set mediapipe model

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 10.0, (int(cap.get(3)), int(cap.get(4))))


with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # Flag variable to signal breaking out of all loops
    break_all_loops = False

    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            video_file_name = f'{VIDEO_PATH}/{action}/{sequence}.avi'
            out = cv2.VideoWriter(video_file_name, fourcc, 10.0, (int(cap.get(3)), int(cap.get(4))))


            for frame_num in range(sequence_length):
                if break_all_loops:
                    break

                ret, frame = cap.read()
                if not ret:
                    print("Failed to capture frame")

                    break  # If no frame is read, break out of the loop
                



                image, results = mediapipe_detection(frame, holistic)

                # write in the video file
                out.write(image)


                draw_styled_landmarks(image, results)

                if frame_num == 0:
                    for countdown in range(2, 0, -1):
                        cv2.putText(image, f'GET READY FOR "{action}" in {countdown}', (30,100),
                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0, 0), 4, cv2.LINE_AA)
                        cv2.imshow('OpenCV Feed', image)
                        cv2.waitKey(1000)
                        image = frame.copy()  # Reset image to clean frame
                else:
                    cv2.putText(image, f'RECORDING [{sequence + 1}/{no_sequences}] "{action}" - Frame {frame_num + 1}/{sequence_length}', (30,100),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0, 255), 4, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('t'):
                    break
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break_all_loops = True
                    break

            out.release()  # This should be here, inside the sequence loop but outside the frame loop

        if break_all_loops:
            break

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)


I0000 00:00:1715347541.301650       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M1


: 

In [ ]:
cap.release()
cv2.destroyAllWindows()